In [32]:
from pathlib import Path
from typing import Optional, Iterator

from docling.datamodel.accelerator_options import AcceleratorOptions, AcceleratorDevice
from docling_core.transforms.chunker import BaseChunker, PageChunker
from docling_core.types import DoclingDocument
from langchain_docling.loader import BaseMetaExtractor, MetaExtractor
from langchain_core.documents import Document

from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TesseractCliOcrOptions

DEFAULT_EXTENSIONS = ["."+ fmt.value for fmt in InputFormat]


def collect_documents_paths(
        directory_path: Optional[str] = None,
        recursive: bool = True,
        file_extensions: Optional[list[str]] = None
) -> list[Path]:
    if file_extensions is None:
        file_extensions = DEFAULT_EXTENSIONS

    if not directory_path:
        raise ValueError("Either file_path, directory_path, or config.FILE_PATH must be provided")

    dir_path = Path(directory_path)
    if not dir_path.exists() or not dir_path.is_dir():
        raise ValueError(f"Invalid directory path: {directory_path}")

    return _collect_files(dir_path, file_extensions, recursive)


def chunk(document_list: Iterator[DoclingDocument],
          chunker: Optional[BaseChunker] = PageChunker,
          meta_extractor: Optional[BaseMetaExtractor] = MetaExtractor()) \
        -> Iterator[Document]:
    for doc in document_list:
        chunk_iter = chunker.chunk(doc)
        for _chunk in chunk_iter:
            yield Document(
                page_content=chunker.contextualize(chunk=_chunk),
                metadata=meta_extractor.extract_chunk_meta(
                    file_path=doc.name,  # todo change this later
                    chunk=_chunk,
                ),
            )

def convert_to_markdown(documents: list[Path]) -> Iterator[DoclingDocument]:
    if not documents:
        raise ValueError("Documents list cannot be empty")

    # Create document converter with format-specific options
    converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=_get_pdf_pipline())
        }
    )

    for doc in documents:
        try:
            result = converter.convert(doc)

            if not result.errors:
                yield result.document

        except Exception:
            continue


def _get_pdf_pipline() -> PdfPipelineOptions:
    pipeline_options = PdfPipelineOptions()
    pipeline_options.do_ocr = True
    pipeline_options.do_table_structure = True
    pipeline_options.table_structure_options.do_cell_matching = True
    pipeline_options.generate_picture_images = True
    pipeline_options.generate_table_images = True
    pipeline_options.ocr_options = TesseractCliOcrOptions()

    pipeline_options.accelerator_options = AcceleratorOptions(
        num_threads=4, device=AcceleratorDevice.AUTO
    )
    return pipeline_options


def _collect_files(directory: Path, extensions: list[str], recursive: bool) -> list[Path]:
    files = []
    pattern_func = directory.rglob if recursive else directory.glob
    for ext in extensions:
        files.extend(pattern_func(f"*{ext}"))
    return files


In [57]:
from typing import Optional
from langchain_core.documents import Document
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_milvus import Milvus

from langchain_core.embeddings import Embeddings


class VectorStore:

    def __init__(
            self,
            embedding: Embeddings,
            collection_name: str = "docling_demo",
            drop_old: bool = False,
    ) -> None:

        # consistency_level = "Session"
        # auto_id = True

        self.embedding = embedding
        self.collection_name: str = collection_name
        self.vectorstore: Optional[Milvus] = None
        # self.vectorstore = Milvus(
        #     embedding_function=embedding,
        #     collection_name=self.collection_name,
        #     connection_args={
        #         "host": config.VECTOR_HOST,
        #         "port": config.VECTOR_PORT,
        #     },
        #     index_params={"index_type": "FLAT"},
        #     drop_old=drop_old,
        #     consistency_level=consistency_level,
        #     auto_id=auto_id,
        # )

    def create_from_documents(
        self,
        documents: list[Document],
        drop_old: bool = True
    ) -> Milvus:
        if not documents:
            raise ValueError("Documents list cannot be empty")

        self.vectorstore = Milvus.from_documents(
            documents=documents,
            embedding=self.embedding,
            collection_name=self.collection_name,
            connection_args={"host": "localhost", "port": "19530"},
            index_params={
                "index_type": "FLAT",
                "metric_type": "IP"  # use "L2" for Euclidean
            },
            drop_old=drop_old,
        )
        return self.vectorstore


    def add_documents(self, documents: list[Document]) -> list[str]:
        if not self.vectorstore:
            raise RuntimeError(
                "Vector store not initialized. Call create_from_documents() first."
            )

        if not documents:
            raise ValueError("Documents list cannot be empty")

        return self.vectorstore.add_documents(documents)

    def get_retriever(self, top_k: int = 5) -> VectorStoreRetriever:
        if not self.vectorstore:
            raise RuntimeError(
                "Vector store not initialized. Call create_from_documents() first."
            )

        return self.vectorstore.as_retriever(search_kwargs={"k": top_k})

    def similarity_search(
            self,
            query: str,
            k: int = 5
    ) -> list[Document]:
        if not self.vectorstore:
            raise RuntimeError(
                "Vector store not initialized. Call create_from_documents() first."
            )

        return self.vectorstore.similarity_search(query, k=k)


In [3]:
# from typing import Optional, List
# import faiss
# import numpy as np
# from langchain.docstore.document import Document
# from langchain_community.vectorstores import FAISS
# from langchain.embeddings.base import Embeddings
# from langchain.vectorstores.base import VectorStoreRetriever
#
#
# class VectorStore:
#     def __init__(
#         self,
#         embedding: Embeddings,
#         collection_name: str = "faiss_demo",
#         drop_old: bool = False,
#     ) -> None:
#         self.embedding = embedding
#         self.collection_name = collection_name
#         self.vectorstore: Optional[FAISS] = None
#         self.drop_old = drop_old
#
#         if drop_old:
#             # Start fresh
#             self.vectorstore = None
#
#     def create_from_documents(self, documents: List[Document]) -> FAISS:
#         """
#         Create a FAISS index from a list of documents.
#         """
#         if not documents:
#             raise ValueError("Documents list cannot be empty")
#
#         # Build FAISS index
#         self.vectorstore = FAISS.from_documents(
#             documents=documents,
#             embedding=self.embedding
#         )
#
#         return self.vectorstore
#
#     def add_documents(self, documents: List[Document]) -> List[str]:
#         """
#         Add new documents to an existing FAISS index.
#         """
#         if not self.vectorstore:
#             raise RuntimeError(
#                 "Vector store not initialized. Call create_from_documents() first."
#             )
#
#         if not documents:
#             raise ValueError("Documents list cannot be empty")
#
#         return self.vectorstore.add_documents(documents)
#
#     def get_retriever(self, top_k: int = 5) -> VectorStoreRetriever:
#         """
#         Return a retriever object for semantic search.
#         """
#         if not self.vectorstore:
#             raise RuntimeError(
#                 "Vector store not initialized. Call create_from_documents() first."
#             )
#
#         return self.vectorstore.as_retriever(search_kwargs={"k": top_k})
#
#     def similarity_search(self, query: str, k: int = 5) -> List[Document]:
#         """
#         Perform similarity search on the FAISS index.
#         """
#         if not self.vectorstore:
#             raise RuntimeError(
#                 "Vector store not initialized. Call create_from_documents() first."
#             )
#
#         return self.vectorstore.similarity_search(query, k=k)
#
#     def save_local(self, path: str) -> None:
#         """
#         Save FAISS index to disk.
#         """
#         if not self.vectorstore:
#             raise RuntimeError("No FAISS index to save.")
#
#         self.vectorstore.save_local(path)
#
#     def load_local(self, path: str) -> None:
#         """
#         Load FAISS index from disk.
#         """
#         self.vectorstore = FAISS.load_local(
#             path=path,
#             embeddings=self.embedding,
#             allow_dangerous_deserialization=True,
#         )
#

2025-10-16 03:57:54,531 - INFO - Loading faiss with AVX2 support.
2025-10-16 03:57:54,606 - INFO - Successfully loaded faiss with AVX2 support.


In [52]:
from typing import Optional
from langchain_huggingface.embeddings import HuggingFaceEmbeddings


class EmbeddingModel:

    def __init__(self, model_id: Optional[str] = None) -> None:
        self.model_id: str = "sentence-transformers/all-MiniLM-L6-v2"
        self.embedding: HuggingFaceEmbeddings = HuggingFaceEmbeddings(
            model_name=self.model_id
        )

    def get_embedding(self) -> HuggingFaceEmbeddings:
        return self.embedding

In [58]:
from typing import Optional, Iterator

from colorama import Fore, Style, init
from docling_core.transforms.chunker.hybrid_chunker import HybridChunker
from langchain_huggingface import HuggingFaceEmbeddings

# Initialize colorama
init(autoreset=True)


def _print_step(emoji: str, message: str, color: str = Fore.CYAN):
    """Print a colored step message."""
    print(f"{color}{emoji} {message}{Style.RESET_ALL}")


def _print_success(emoji: str, message: str):
    """Print a success message."""
    print(f"{Fore.GREEN}{emoji} {message}{Style.RESET_ALL}")


def _print_info(emoji: str, message: str):
    """Print an info message."""
    print(f"{Fore.YELLOW}{emoji} {message}{Style.RESET_ALL}")


def ingest_pipeline(
        directory_path: Optional[str] = None,
        recursive: bool = True,
        file_extensions: Optional[list[str]] = None,
        batch_size: int = 100
) -> VectorStore:
    """
    Complete ingestion pipeline: collect, convert, chunk, embed, and store documents.

    Args:
        directory_path: Path to directory containing documents
        recursive: Whether to search subdirectories
        file_extensions: List of file extensions to process
        batch_size: Number of chunks to collect before adding to vector store

    Returns:
        VectorStore instance with embedded documents
    """
    print(f"\n{Fore.MAGENTA}{'='*70}{Style.RESET_ALL}")
    _print_step("🚀", "Starting Document Ingestion Pipeline", Fore.MAGENTA)
    print(f"{Fore.MAGENTA}{'='*70}{Style.RESET_ALL}\n")

    # Step 1: Collect document paths
    _print_step("📁", f"Collecting documents from: {directory_path}", Fore.CYAN)
    _print_info("  ℹ️", f"Recursive search: {recursive}")
    if file_extensions:
        _print_info("  ℹ️", f"File extensions: {', '.join(file_extensions)}")

    paths = collect_documents_paths(directory_path, recursive, file_extensions)
    _print_success("  ✅", f"Found {len(paths)} document(s)")

    # Step 2: Convert documents to markdown
    _print_step("📄", "Converting documents to markdown format...", Fore.CYAN)
    iter_document = convert_to_markdown(paths)

    # Step 3: Chunking
    _print_step("✂️", "Chunking documents with HybridChunker...", Fore.CYAN)
    _print_info("  ℹ️", f"Batch size: {batch_size} chunks")
    chunker = HybridChunker()
    chunk_iter = chunk(iter_document, chunker)

    # Step 4: Initialize embeddings and vector store
    _print_step("🧠", "Initializing embedding model...", Fore.CYAN)
    model_name = "all-MiniLM-L6-v2"
    device = "cpu"

    embedding = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True}
    )

    _print_success("  ✅", "Embedding model loaded")

    _print_step("🗄️", "Creating vector store...", Fore.CYAN)
    vector_store = VectorStore(embedding)
    _print_success("  ✅", "Vector store initialized")

    # Step 5: Add chunks to vector store in batches
    _print_step("💾", "Adding chunks to vector store in batches...", Fore.CYAN)

    batch = []
    total_chunks = 0
    batch_count = 0

    for doc in chunk_iter:
        if total_chunks == 0:
            temp = [doc]
            vector_store.create_from_documents(temp)
        batch.append(doc)
        total_chunks += 1

        if len(batch) >= batch_size:
            batch_count += 1
            _print_info("  📦", f"Processing batch {batch_count} ({len(batch)} chunks)...")
            vector_store.add_documents(batch)
            _print_success("  ✅", f"Batch {batch_count} added successfully")
            batch = []

    # Add any remaining chunks
    if batch:
        batch_count += 1
        _print_info("  📦", f"Processing final batch ({len(batch)} chunks)...")
        vector_store.add_documents(batch)
        _print_success("  ✅", f"Final batch added successfully")

    # Summary
    print(f"\n{Fore.GREEN}{'='*70}{Style.RESET_ALL}")
    _print_success("🎉", "Ingestion Pipeline Completed Successfully!")
    print(f"{Fore.GREEN}{'='*70}{Style.RESET_ALL}")
    _print_info("📊", f"Total documents processed: {len(paths)}")
    _print_info("📊", f"Total chunks created: {total_chunks}")
    _print_info("📊", f"Total batches processed: {batch_count}")
    print()

    return vector_store




In [55]:
directory = "test"

In [59]:
store = ingest_pipeline(directory)


🚀 Starting Document Ingestion Pipeline

📁 Collecting documents from: test
  ℹ️ Recursive search: True
  ✅ Found 804 document(s)
📄 Converting documents to markdown format...
✂️ Chunking documents with HybridChunker...
  ℹ️ Batch size: 100 chunks
🧠 Initializing embedding model...


2025-10-16 04:48:56,620 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


  ✅ Embedding model loaded
🗄️ Creating vector store...
  ✅ Vector store initialized
💾 Adding chunks to vector store in batches...


2025-10-16 04:48:59,520 - INFO - detected formats: [<InputFormat.DOCX: 'docx'>]
2025-10-16 04:48:59,535 - INFO - Going to convert document batch...
2025-10-16 04:48:59,536 - INFO - Initializing pipeline for SimplePipeline with options hash 995a146ad601044538e6a923bea22f4e
2025-10-16 04:48:59,537 - INFO - Processing document Content_Assets_20240722IRF.docx
2025-10-16 04:48:59,549 - INFO - deleted item in tree at stack: (1,) => #/texts/0
2025-10-16 04:48:59,556 - INFO - deleted item in tree at stack: (1,) => #/texts/0
2025-10-16 04:48:59,561 - INFO - deleted item in tree at stack: (1,) => #/texts/0
2025-10-16 04:48:59,577 - INFO - deleted item in tree at stack: (6,) => #/texts/9
2025-10-16 04:48:59,585 - INFO - deleted item in tree at stack: (8,) => #/texts/13
2025-10-16 04:48:59,607 - INFO - deleted item in tree at stack: (10,) => #/texts/28
2025-10-16 04:48:59,610 - INFO - deleted item in tree at stack: (10,) => #/texts/28
2025-10-16 04:48:59,620 - INFO - deleted item in tree at stack:

  📦 Processing batch 1 (100 chunks)...
  ✅ Batch 1 added successfully


...

  📦 Processing batch 92 (100 chunks)...
  ✅ Batch 92 added successfully


2025-10-16 05:02:44,980 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-16 05:02:44,984 - INFO - Going to convert document batch...
2025-10-16 05:02:44,985 - INFO - Processing document Content_Files_JeanAkiki_06.pdf
2025-10-16 05:02:56,537 - INFO - Finished converting document Content_Files_JeanAkiki_06.pdf in 11.56 sec.
2025-10-16 05:02:56,779 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-16 05:02:56,781 - INFO - Going to convert document batch...
2025-10-16 05:02:56,782 - INFO - Processing document Content_Files_JeanAkiki_08.pdf
2025-10-16 05:03:02,981 - INFO - Finished converting document Content_Files_JeanAkiki_08.pdf in 6.21 sec.


  📦 Processing batch 93 (100 chunks)...


2025-10-16 05:06:39,517 [ERROR][handler]: RPC error: [insert_rows], <MilvusException: (code=<bound method _InactiveRpcError.code of <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv6:%5B::1%5D:19530: Failed to connect to remote host: connect: Connection refused (111)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNKNOWN: ipv6:%5B::1%5D:19530: Failed to connect to remote host: connect: Connection refused (111)"}"
>>, message=[insert_rows] Retry run out of 75 retry times, message=failed to connect to all addresses; last error: UNKNOWN: ipv6:%5B::1%5D:19530: Failed to connect to remote host: connect: Connection refused (111))>, <Time:{'RPC start': '2025-10-16 05:03:07.823892', 'RPC error': '2025-10-16 05:06:39.517450'}> (decorators.py:140)
2025-10-16 05:06:39,520 - ERROR - Failed to insert batch s

MilvusException: <MilvusException: (code=<bound method _InactiveRpcError.code of <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv6:%5B::1%5D:19530: Failed to connect to remote host: connect: Connection refused (111)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNKNOWN: ipv6:%5B::1%5D:19530: Failed to connect to remote host: connect: Connection refused (111)"}"
>>, message=[insert_rows] Retry run out of 75 retry times, message=failed to connect to all addresses; last error: UNKNOWN: ipv6:%5B::1%5D:19530: Failed to connect to remote host: connect: Connection refused (111))>